# Transfer Learning using a Pre-trained Transformer Model for Text Summarization

**Approach**

In this notebook, we solve a natural language processing (NLP) task of text summarization, by using an **encoder-decoder** Transformer architecture-based pre-trained model T5. Specifically, the T5 model is fine-tuned for transfer learning. The pre-trained T5 model is obtained from the Huggingface library.


- Step 1: Load the raw dataset
- Step 2: Tokenize the raw dataset
- Step 3: Instantiate a pre-trained model from the model checkpoint
- Step 4: Create a data collator
- Step 5: Create train and test dataset loader objects
- Step 6: Compile the Model
- Step 7: Train the model
- Step 8: Test the model by generating summary of a sample text



**Dataset**

BillSum is a text summarization dataset consisting of the summarization of US Congressional and California state bills. It has 1237 samples each containing the bill text ("text"), a  summary of the bill ("summary"), and the title of the bill ("title").


**Acknowledgmentgement**

This notebook is adapted from the following resources.
- https://huggingface.co/learn/nlp-course/chapter7/5?fw=tf
- https://huggingface.co/docs/transformers/tasks/summarization
- https://github.com/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb
- https://keras.io/examples/nlp/t5_hf_summarization/



In [1]:
! pip install transformers datasets
! pip install rouge-score
! pip install huggingface_hub
! pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=98ac6be0a991b451e8435057c04f45d2ae94411b9d224b284459133e68f764e6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73

**Store the Fine-Tuned Model on the Hugging Face Hub**

We can store the fine-tuned model Hugging Face Hub cloud repository. This will make it easier to reuse the fine-tuned model.

We will use push_to_hub API for this purpose. However, to use this utility, we need to have a Hugging Face account (sign up with a Hugging Face account at: https://huggingface.co/welcome). Then, get an authentication token and input the token after running the following cell.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import os
import logging

import random
from IPython.display import display, HTML

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

import tensorflow as tf
import keras_nlp

import datasets
from datasets import load_dataset

import transformers
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import TFAutoModelForSeq2SeqLM
from transformers import create_optimizer, AdamWeightDecay


# Only log error messages
# tf.get_logger().setLevel(logging.ERROR)

# os.environ["TOKENIZERS_PARALLELISM"] = "false"


print(transformers.__version__)


Using TensorFlow backend
4.34.0


**Useful Variables**

In [4]:
TRAIN_TEST_SPLIT = 0.2 # Percentage of the dataset for evaluation
MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output from the model

# The t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

**Step 1: Load the raw dataset**

The load_dataset method returns a dictionary object of type DatasetDict.


In [5]:
raw_datasets=load_dataset("billsum", split="ca_test")
raw_datasets


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [6]:
# Split the dataset into a train and test set with the train_test_split method

raw_datasets = raw_datasets.train_test_split(test_size=TRAIN_TEST_SPLIT)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [7]:

# Access a sample of the training raw_datasets object by indexing
raw_train_dataset = raw_datasets["train"]
print(raw_train_dataset[0])

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nArticle 3 (commencing with Section 115810) is added to Chapter 4 of Part 10 of Division 104 of the Health and Safety Code, to read:\nArticle  3. The Consideration of Alternatives for Artificial Turf Infill Act of 2016\n115810.\nThe Legislature finds and declares all of the following:\n(a) Thousands of schools, parks, and local governments have installed artificial turf fields throughout the state. It has allowed them to use fields year round, save water, and save money, among other benefits.\n(b) Not all artificial turf fields are made from the same materials. While most artificial turf fields use less expensive crumb rubber infill from groundup used car and truck tires, many companies now offer artificial turf infill alternatives made from coconut fibers, rice husks, cork, sand, or virgin crumb rubber. Organic alternative infills can help reduce synthetic turf field temperatures on hot days by as muc

In [8]:
# Inspect the features of the raw_train_dataset object
raw_train_dataset.features

{'text': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

**Display some random samples from the dataset**

In [9]:
# A function to display some random samples from the dataset
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


# Display some random samples from the training dataset
show_random_elements(raw_datasets["train"])

**Step 2: Tokenize the raw dataset**

Tokenization is the process of converting the text to numbers
This will be done via a tokenizer object.

- Step 2(a): Instantiate a tokenizer object
- Step 2(b): Define a function to preprocess and tokenize the input
- Step 2(c): Tokenize the batches


In [10]:
'''
Step 2(a): Instantiate a tokenizer object
This is done by using a suitable model checkpoint
'''
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)



'''
Step 2(b): Define a function to preprocess and tokenize the input

- Preprocess: Prefix the input with a prompt so T5 knows this is a summarization task.
  Some models capable of multiple NLP tasks require prompting for specific tasks.

- Tokenize
- Apply truncation to both the input text and their labels (summaries).


The truncation=True argument of the tokenizer object will truncate the sequences
    that are longer than the model max length
    (e.g., 512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

For truncating the sequences that are longer than the specified max length,
use the following code:
tokenizer(sequences, max_length=8, truncation=True)

Use the as_target_tokenizer() method of the tokenizer object
  to tokenize the labels in parallel to the inputs.

'''

# Prefix the input with a prompt so T5 knows this is a summarization task
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs,
                             max_length=MAX_INPUT_LENGTH,
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"],
                           max_length=MAX_TARGET_LENGTH,
                           truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs




'''
- Step 2(c): Tokenize the batches

We want to keep the data as a dataset object after tokenization.
So, we use the Dataset map() method for tokenization,
by enabling it to utilize the tokenizer function.
The map() method works by applying a function to each element of the dataset.
This gives us the flexibility to apply additional preprocessing
via the map() method.

We set batched=True load samples in the RAM in batches.
This is possible because the datasets from the 🤗 Datasets library
are stored on the disk as they are Apache Arrow files.

Note that we didn't pad the samples.
Because it's more efficient to apply padding during the creation of the batches.
In such a case, we only need to pad to the maximum length in that batch,
and not the maximum length in the entire dataset.
This will save time and processing power
when the inputs have very variable lengths!
'''
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets



Map:   0%|          | 0/989 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/248 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

In [11]:
# Inspect a preprocessed sample
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[21603, 10, 37, 151, 13, 8, 1015, 13, 1826, 103, 3, 35, 2708, 38, 6963, 10, 180, 3073, 9562, 1300, 7491, 220, 41, 287, 526, 4733, 28, 5568, 850, 3449, 16968, 19, 974, 12, 8647, 314, 13, 2733, 335, 13, 6022, 3, 15442, 13, 8, 1685, 11, 6859, 3636, 6, 12, 608, 10, 7491, 1877, 37, 9151, 257, 13, 13661, 7, 21, 24714, 7538, 89, 86, 16326, 1983, 13, 1421, 850, 3449, 10415, 37, 28204, 12902, 11, 15884, 7, 66, 13, 8, 826, 10, 41, 9, 61, 3, 29541, 13, 2061, 6, 9307, 6, 11, 415, 10524, 43, 3029, 7353, 23802, 4120, 1019, 8, 538, 5, 94, 65, 2225, 135, 12, 169, 4120, 215, 1751, 6, 1097, 387, 6, 11, 1097, 540, 6, 859, 119, 1393, 5, 41, 115, 61, 933, 66, 7353, 23802, 4120, 33, 263, 45, 8, 337, 1397, 5, 818, 167, 7353, 23802, 4120, 169, 705, 2881, 5764, 5937, 7859, 16, 16326, 45, 1591, 413, 261, 443, 11, 4072, 12541, 6, 186, 688, 230, 462, 7353, 23802, 16, 16326, 10336, 263, 45, 9417, 6433, 7, 6, 6605, 3, 11823, 157, 7, 6, 4301, 157, 6, 3, 7, 232, 6, 42, 24556, 5764, 5937, 7859, 5, 15045

**Hyperparameters**

In [12]:
BATCH_SIZE = 8

MAX_EPOCHS = 4

'''
The initial learning rate is used by the optimizers, e.g., SGD, ADAM, NADAM, etc.

Note that transformer models benefit from a much lower learning rate than the default for Adam, which is 1e-3,
A much smaller rate, e.g., 5e-5, is a better starting point.
'''
INITIAL_LEARNING_RATE = 2e-5

WEIGHT_DECAY = 0.01


**Step 3: Instantiate a pre-trained model from the model checkpoint**


In [13]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


**Step 4: Create a data collator**

Put together the samples in a batch by using a collate function.
By default, this function converts samples to tf.Tensor and
concatenate them (recursively if the elements are lists,
tuples, or dictionaries).

We can't utilize the default function as our inputs have variable lengths.
To address the variable-length issue, we will apply padding during the batching.
This is an efficient approach (padding during baching) as we can
avoid having over-long inputs with a lot of padding.

Note that we need a special kind of data collator, which will not only pad the inputs
to the maximum length in the batch, but also the labels.

For applying the correct amount of padding to the items
of the input and labels in a batch, we use DataCollatorForSeq2Seq.

Set the return_tensors='np' argument to get the output as a NumPy array.
We use "np" instead of "tf", because "np" is more reliable and performant.
The TF dataset pipeline uses a NumPy loader internally.


In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")


'''
Define another data collator to use during the the training via a callback function
for generating evaluation text data so that we can compute the ROGUE score and use it as a training metric.
The ROGUE metrics require to generate text from the model.
To speed things up, we can compile our generation loop with XLA.
This results in a huge speedup - up to 100X!
The downside of XLA generation, though, is that it doesn't like variable input shapes,
because it needs to run a new compilation for each new input shape!
To compensate for that, let's use pad_to_multiple_of for the dataset we use for text generation.
This will reduce the number of unique input shapes a lot, meaning we can get
the benefits of XLA generation with only a few compilations.

Note:
pad_to_multiple_of (int, optional) — If set will pad the sequence to a multiple of the provided value.
'''
generation_data_collator = DataCollatorForSeq2Seq(tokenizer,
                                                  model=model,
                                                  return_tensors="np",
                                                  pad_to_multiple_of=128)



**Step 5: Create train and test dataset loader objects**

Create the train and validation dataset by putting togerher the
tokenized dataset and collated dataset via the model prepare_tf_dataset() method.

It will wrap a tf.data.Dataset around the dataset, with an optional collation function.

The tf.data.Dataset is a native TensorFlow format that Keras can use for model.fit().


In [15]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,
)

tf_validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


**Step 6: Compile the Model**

Before we instantiate and compile the model, we need to define the optimizer and the loss function.


In [16]:
'''
Reset all states generated by Keras.
It deletes the TensorFlow graph before creating a new model,
otherwise memory overflow will occur.
'''
tf.keras.backend.clear_session()

'''
To reproduce the same result by the model in each iteration, we use fixed seeds for random number generation.
'''
# np.random.seed(42)
# tf.random.set_seed(42)



###################### Optimizer ##########################
# We provide various choices for the optimizer

'''
For the learning schedule, we need to set how long training is going to be, i.e., the number of training steps.
num_of_training_steps = (num_of_training_samples // batch_size) *  epochs

Since the tf_train_dataset is batched, its len() is already num_of_training_samples // batch_size
'''
num_of_training_steps = len(tf_train_dataset) * MAX_EPOCHS

'''
Scheduler: ExponentialDecay
'''
# lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=INITIAL_LEARNING_RATE,
#     decay_steps=num_of_training_steps,
#     decay_rate=WEIGHT_DECAY,
#     staircase=True)

'''
Scheduler: PolynomialDecay
'''
lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=INITIAL_LEARNING_RATE,
    decay_steps=num_of_training_steps,
    end_learning_rate=0.0,
    power=1.0,
    cycle=False,
    name=None
)

'''
Optimizer:

Instantiate an optimizer. Use one of the following choices.
- Fixed LR: learning_rate=INITIAL_LEARNING_RATE
- Scheduled LR: learning_rate=lr_scheduler
'''
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr_scheduler, momentum=0.9, nesterov=False)
#optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
#optimizer=tf.keras.optimizers.Nadam(learning_rate=lr_scheduler)
#optimizer=tfa.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY)
#optimizer=tfa.optimizers.LAMB(learning_rate=lr_scheduler, weight_decay_rate=WEIGHT_DECAY)

optimizer = AdamWeightDecay(learning_rate=INITIAL_LEARNING_RATE, weight_decay_rate=WEIGHT_DECAY)

# optimizer, schedule = create_optimizer(
#     init_lr=INITIAL_LEARNING_RATE,
#     num_warmup_steps=0,
#     num_train_steps=num_train_steps,
#     weight_decay_rate=WEIGHT_DECAY,
# )




###################### Compile the Model ##########################

'''
No loss specified in compile().
The model's internal loss computation will be used as the loss.
This is a common way to train TensorFlow models in Transformers!
'''
model.compile(optimizer=optimizer)
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Evaluate the training performance using the ROUGE metric from Keras NLP**

- ROUGE-L is a score based on the length of the longest common subsequence
in the reference text and the hypothesis text.

- ROUGE-N is a score based on the number of matching n-grams between
the reference text and the hypothesis text.


In [17]:
rouge_l = keras_nlp.metrics.RougeL()

'''
A function to calculate the ROUGE metric using the predictions and labels
'''
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute the rogue score
    result = rouge_l(decoded_labels, decoded_preds)
    #use_stemmer=True

    # Disply the F1 score; other options are "precision", "recall"
    result = {"RougeL": result["f1_score"]}

    return result

**Callback Functions**

Define the following callback functions.
- PushToHubCallback
- KerasMetricCallback



In [18]:
'''
PushToHubCallback
It will sync up the fine-tuned model with the Hugging Face Hub.
First, the model will be stored (serialized) on the disk (output_dir).
Then, it will be synced.

This function will allow model reuse.
- The locally stored model can be loaded from "output_dir"
- The cloud-stored model can be loaded from the Hub

The function will allow to resume training from other machines,
share the model after training is finished,
and even test the model's inference quality midway through training!
'''

# Define a name of the fine-tuned model for the callback function
model_name = MODEL_CHECKPOINT.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-summarization-billsum"


push_to_hub_callback = PushToHubCallback(
    output_dir="./model_summarization_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)


'''
KerasMetricCallback is a callback for computing advanced metrics.
There are a number of common metrics in NLP like ROUGE which are hard to fit into
our compiled training loop because they depend on decoding predictions and labels
back to strings with the tokenizer, and calling arbitrary Python functions to compute the metric.
The KerasMetricCallback will wrap a metric function, outputting metrics as training progresses.
'''
metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=generation_dataset,
    predict_with_generate=True, use_xla_generation=True
)

callbacks = [metric_callback, push_to_hub_callback]

Cloning https://huggingface.co/hasan-mr/t5-small-finetuned-summarization-billsum into local empty directory.


Download file tf_model.h5:   0%|          | 15.4k/357M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/357M [00:00<?, ?B/s]

**Step 7: Train the model**


In [19]:
'''
Train in mixed-precision float16
Mixed precision is the use of both 16-bit and 32-bit floating-point types
in a model during training to make it run faster and use less memory.
'''
tf.keras.mixed_precision.set_global_policy("mixed_float16")


# Fine-tune the model
model.fit(tf_train_dataset,
          validation_data=tf_validation_dataset,
          epochs=MAX_EPOCHS,
          callbacks=callbacks)


Epoch 1/4
123/123 [==============================] - ETA: 0s - loss: 3.3746

/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  return py_builtins.overload_of(f)(*args)


123/123 [==============================] - 210s 2s/step - loss: 3.3746 - val_loss: 2.7009 - RougeL: 0.1060
Epoch 2/4
123/123 [==============================] - 166s 1s/step - loss: 2.8289 - val_loss: 2.5104 - RougeL: 0.1088
Epoch 3/4
123/123 [==============================] - 171s 1s/step - loss: 2.6672 - val_loss: 2.4200 - RougeL: 0.1210
Epoch 4/4
123/123 [==============================] - 172s 1s/step - loss: 2.5791 - val_loss: 2.3660 - RougeL: 0.1299


**Step 8: Test the model by generating summary of a sample text**

We will use the fine-tuned model to summarize a sample text by using 3 approaches.
- Use the current fine-tuned model
- Load the saved fine-tuned model from the disk
- Load the saved fine-tuned model from the Hugging Face Hub

**Step 8 (a): Use the current fine-tuned model**

In [20]:
# A sample text for summarization
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

# Tokenize the text
inputs = tokenizer(text, return_tensors="tf").input_ids

# Generate output token ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

# Generate output text by decoding the token ids
tokenizer.decode(outputs[0], skip_special_tokens=True)

"The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history. It'll ask the ultra-wealthy and corporations to pay their fair share."

**Step 8 (b): Load the saved fine-tuned model from the disk**

In [ ]:
# Path to the saved model on the disk
output_dir="./model_summarization_save"

# Load tokenizer and model from the saved model on the disk
tokenizer_saved_model = AutoTokenizer.from_pretrained(output_dir)
saved_model = TFAutoModelForSeq2SeqLM.from_pretrained(output_dir)

# A sample text for summarization
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

# Tokenize the text
inputs_saved_model = tokenizer_saved_model(text, return_tensors="tf").input_ids

# Generate output token ids
outputs_saved_model = saved_model.generate(inputs_saved_model, max_new_tokens=100, do_sample=False)

# Generate output text by decoding the token ids
tokenizer_saved_model.decode(outputs_saved_model[0], skip_special_tokens=True)


**Step 8 (c): Load the saved fine-tuned model from the Hugging Face Hub**

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# Fine-tuned model stored on the HUgging Face Hub
FINE_TUNED_MODEL = "hasan-mr/t5-small-finetuned-summarization-billsum"

# Load tokenizer and model from the Hugging Face Hub
tokenizer_hgg = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL)
model_hgg = TFAutoModelForSeq2SeqLM.from_pretrained(FINE_TUNED_MODEL)

# A sample text for summarization
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

# Tokenize the text
inputs_hgg = tokenizer_hgg(text, return_tensors="tf").input_ids

# Generate output token ids
outputs_hgg = model_hgg.generate(inputs_hgg, max_new_tokens=100, do_sample=False)

# Generate output text by decoding the token ids
tokenizer_hgg.decode(outputs_hgg[0], skip_special_tokens=True)
